$\textbf{UNAM - FES Acatlán}$

$\textbf{Ciencia de Datos}
\\
\textbf{Visualización de la Información}$

$\textbf{Laboratorio de Visualización}
\\
\textbf{Proyecto final 2020: Animación en la Visualización de Datos, segunda parte} 
\\
\textbf{Profesor: Rubén Romero Ruiz}
\\
\textbf{Nombre: González Paz Edgar} $

$\textbf{Objetivo}$

Crear dos visualizaciones animadas mediante dos diferentes bibliotecas de Python, que sinteticen un conjunto de datos, seleccionando la técnica de visualización más adecuada para la representación de la información y esquematizar conocimiento utilizando imágenes en movimiento.

Las bibliotecas utilizadas serán: Matplotlib y Plotly

El dataset utilizado será $\textbf{International football results from 1872 to 2020}$, una recopilación de
resultados de partidos de fútbol soccer de equipos nacionales.

Antes de realizar las dos visualizaciones, desplegamos el dataset para poder reconocer cada atributo para así poder hacer
nuevos dataframes para crear las visualizaciones: la primera un mapa corroplético del número total de partidos ganados en toda su historia y la segunda es una gráfica de barras que nos mostrará un top 10 de los países donde más se jugaron juegos.

Usé la librería pandas para manipular el dataset.

In [1]:
import pandas as pd
df = pd.read_csv('results.csv')
df.head(5)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [2]:
#Número de registros
df.shape[0]

41586

### Segunda visualización

Generé primero el dataframe para crear la visualización.

In [3]:
n=df.shape[0]
home_team_win = []
for i in range(n):
    if df.home_score[i] > df.away_score[i]:
        home_team_win.append(1)
    else:
        home_team_win.append(0)
        
away_team_win = []
for i in range(n):
    if df.away_score[i] > df.home_score[i]:
        away_team_win.append(1)
    else:
        away_team_win.append(0)
df['home_team_win'] = home_team_win
df['away_team_win'] = away_team_win
        
tied_game = []
for i in range(n):
    if df.home_team_win[i] == df.away_team_win[i]:
        tied_game.append(1)
    else:
        tied_game.append(0)
        
df['tied_game'] = tied_game

In [4]:
#Tome sólo los años del atributo fecha y los países donde se celebraron los juegos registrados
df4 = pd.DataFrame({'Year': df.date,'Country': df.country.str.replace(' ','_')})
df4['Year'] = df4['Year'].map(lambda x:x.split('-')[0]) 
df4.head()

,Year,Country
0,1872,Scotland
1,1873,England
2,1874,Scotland
3,1875,England
4,1876,Scotland


In [5]:
#Las concatené para obtener los registros únicos y después poder sumarizarlos
aux = pd.DataFrame(df4.Year + ' ' + df4.Country, columns = ['Combinacion'])
aux.head()

,Combinacion
0,1872 Scotland
1,1873 England
2,1874 Scotland
3,1875 England
4,1876 Scotland


In [6]:
aux = aux['Combinacion'].value_counts()
aux = pd.DataFrame(aux).reset_index()
aux.columns = ['Year+Country', 'Times']
aux.head()

,Year+Country,Times
0,2006 Germany,94
1,1994 United_States,92
2,2010 South_Africa,80
3,1998 France,77
4,2018 Russia,72


In [7]:
Year = aux['Year+Country'].map(lambda x:x.split(' ')[0]) 
Country = aux['Year+Country'].map(lambda x:x.split(' ')[1]) 

df5 = pd.DataFrame({'Country': Country, 'Year': Year, 'Times': aux.Times})
#Ordenamos por años de manera ascendiente
df5 = df5.sort_values(by='Year', ascending=True).reset_index(drop=True)
#Convertimos los años en entero
df5['Year'] = df5['Year'].astype(int)
df5['Country'] = df5['Country'].str.replace('_',' ')
df5.head()

,Country,Year,Times
0,Scotland,1872,1
1,England,1873,1
2,Scotland,1874,1
3,England,1875,1
4,Scotland,1876,2


In [8]:
def Top(current_year):
    dataframe = df5[df5['Year'].eq(current_year)].sort_values(by='Times', ascending=False).head(10).reset_index(drop = True)
    return dataframe

In [9]:
Total_años=[]
for i in range(1872,2021,1):
    Total_años.append(i)

In [10]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline

In [11]:
#Aquí generé una gráfica de barras horizontal
@widgets.interact(Year=Total_años)

def Grafica_TOP10(Year):
    fig, ax = plt.subplots(1, 1, figsize=(8, 6))
    ax.barh(Top(Year)['Country'], Top(Year)['Times'], alpha=0.5, edgecolor='black')
    ax.set_title('Top 10 countries with more games in the year '+str(Year), fontsize = 20)
    ax.set_xlim(0,max(Top(Year)['Times'])+2)
    #Creamos el arreglo parar que la escala del eje de las x tenga intervalo de distancia 2, esto lo tomé porque si el intervalo
    #era de 1, se aplstaban en le eje
    x_ticks=[]
    for i in range(max(Top(Year)['Times'])+2):
        if i % 2 == 0:
            x_ticks.append(i)
    #Ponemos el número correspondiente de partidos        
    for index, value in enumerate(Top(Year)['Times']):
        plt.text(value+0.09, index, str(value))
        
    ax.set_ylabel('Countries', fontsize = 16)
    ax.set_xlabel('Number of matches', fontsize = 16)
    ax.set_xticks(x_ticks)
    ax.grid(True, alpha=0.25)

interactive(children=(Dropdown(description='Year', options=(1872, 1873, 1874, 1875, 1876, 1877, 1878, 1879, 18…

In [12]:
#Si elegimos el año 2017 podemos ver los siguientes datos para corroborar
current_year = 2007
dff = df5[df5['Year'].eq(current_year)].sort_values(by='Times', ascending=False).head(10).reset_index(drop = True)
dff

,Country,Year,Times
0,United States,2007,45
1,Venezuela,2007,37
2,Tanzania,2007,28
3,Greece,2007,27
4,Thailand,2007,25
5,Samoa,2007,24
6,France,2007,21
7,Singapore,2007,18
8,United Arab Emirates,2007,18
9,Malaysia,2007,18


$\textbf{Conclusión}$

La segunda visualización constó de una gráfica de barras comparativa entre el número de partidos jugados por país que puede ser filtrado por año gracias a un menú desplegable que empiezas desde 1872 a 2020, la documentación de estte widget para matplotlib nos permite agregar este elemento fácilmente. Sin embargo, considero que las visualizaciones con la biblioteca Matplotlib resultan más convenientes para gráficas sin interacción, esto toma sentido porque la librería comenzó sólo para elaborar gráficas estáticas, posteriormente se le fueron agregando estos widgets para la interacción cuando se visualice.

Resumiendo, la libería Matplotlib es muy sencilla para generar visualizaciones lo cual tiene como resultado visualizaciones que, en mi punto de vista, no son visualmente atractivas pero aún así logran su cometido, como la presentada en esta segunda parte. 

$\textbf{Referencias}$

Dataset de los resultados de fútbol soccer, disponible en: https://www.kaggle.com/martj42/international-football-results-from-1872-to-2017

Documentación para widgets en Matplotlib, disponible en: https://ipywidgets.readthedocs.io/en/latest/

Documentación para gráfica de barras Matplotlib, disponible en: https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.barh.html

Widgets para Matplotlib, disponible en: https://ipython-books.github.io/33-mastering-widgets-in-the-jupyter-notebook/